<a href="https://colab.research.google.com/github/rromerov/Machine-Learning-Projects/blob/main/Credit_Card_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

Load the data and replace the column names from German to English as indicated on the UCI page. 

In [2]:
file="https://raw.githubusercontent.com/rromerov/Machine-Learning-Projects/main/SouthGermanCredit.asc"
credit=pd.read_table(file,sep=" ")

In [3]:
credit.head()

,laufkont,laufzeit,moral,verw,hoehe,sparkont,beszeit,rate,famges,buerge,...,verm,alter,weitkred,wohn,bishkred,beruf,pers,telef,gastarb,kredit
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


In [4]:
columns_name=["status","duration","credit_history","purpose","amount","savings","employment_duration","installment_rate","personalstatussex","other_debtors","president_residence","property","age","otherinstallmentplans","housing","number_credits","job","people_liable","telephone","foreign_worker","target column"]
credit.columns=columns_name

In [14]:
for i in credit.columns:
  unique_values_per_column=credit[i].unique()
  if (len(unique_values_per_column)>7): #we hide columns that have 7 or more unique values
    pass
  else:  
    print("Column name:",i, sorted(unique_values_per_column))

Column name: status [1, 2, 3, 4]
Column name: credit_history [0, 1, 2, 3, 4]
Column name: savings [1, 2, 3, 4, 5]
Column name: employment_duration [1, 2, 3, 4, 5]
Column name: installment_rate [1, 2, 3, 4]
Column name: personalstatussex [1, 2, 3, 4]
Column name: other_debtors [1, 2, 3]
Column name: president_residence [1, 2, 3, 4]
Column name: property [1, 2, 3, 4]
Column name: otherinstallmentplans [1, 2, 3]
Column name: housing [1, 2, 3]
Column name: number_credits [1, 2, 3, 4]
Column name: job [1, 2, 3, 4]
Column name: people_liable [1, 2]
Column name: telephone [1, 2]
Column name: foreign_worker [1, 2]
Column name: target column [0, 1]


In [6]:
credit.head()

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personalstatussex,other_debtors,...,property,age,otherinstallmentplans,housing,number_credits,job,people_liable,telephone,foreign_worker,target column
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


Partition the data into the 85% training set and the 15% test set. The models will be trained with the cross-validation method, so it is not necessary at this step to generate the validation set. Define as the X variable all input variables and the Y variable as the output variable. 

In [7]:
X=credit.iloc[:,0:-2] #all columns except the last one
y=credit.iloc[:,-1] #only the last column

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=1)